<a href="https://colab.research.google.com/github/cjfghk5697/Classifiviation_BMD/blob/main/bmd_sample_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip3 install timm

In [ ]:
import os
import torch
import csv
import torch.nn as nn
import timm

from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from torchvision.datasets import ImageFolder
from torchvision import transforms

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
test_dir = './test'
model_path = './best.pth'
csv_path = 'submission.csv'

# TEST

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model('resnext101_32x8d', pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc = nn.Linear(1000, 3)

    def forward(self, x):
        return self.fc(self.backbone(x))

In [ ]:
def prediction(net, dataloader, device=device):
    with torch.no_grad():
        result = []
        net.eval()
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            outputs = net(inputs)
            _, preds = torch.max(outputs, 1)
            result.append(preds.item())
    return result

Loading the Test Model

In [ ]:
checkpoint = torch.load(model_path)

net = Model()
net.load_state_dict(checkpoint['model_state_dict'])
net.eval()
net = net.to(device)

Making a dataset

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

test_dataset = ImageFolder(test_dir, transform=test_transform)

In [ ]:
num_workers = 4
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, drop_last=False, num_workers=num_workers)

Prediction

In [ ]:
result = prediction(net, test_loader)

Making a submission file

In [ ]:
with open(csv_path, 'w') as f:
    wr = csv.writer(f)
    wr.writerow(['index', 'label_answer'])
    for idx, prediction in enumerate(result):
        wr.writerow([idx+1, prediction])